In [1]:
import pandas as pd
import numpy as np
import wrds
import re

In [2]:
# We use this everywhere
bea_codes = pd.read_excel('Data/User inputs/NAICS2BEA.xlsx')
bea_seg = pd.read_stata('Data/Temp/levelkey.dta')

# Ind_short to indcode
bea_seg.rename(columns = {'ind_short': 'indcode'}, inplace = True)

# Set up WRDS connection
db = wrds.Connection()

Loading library list...
Done


# Regulation index

All I do here is that I merge industry codes to the regulation index, and compute mean and median by industry group

In [3]:
# Import data
data_reg = pd.read_excel('Data/Raw inputs/regdata_by_3-digit_industry.xls')

# Rename columns, drop some
data_reg.rename(inplace = True, columns = {'industry': 'naics', 
                'Industry-relevant restriction count (Industry Regulation Index)': 'regindex'})
data_reg.drop(columns = ['Industry-relevant words'], inplace = True)

# Create log of reg
data_reg['logreg'] = np.log(data_reg['regindex'])

# Merge with  BEA codes
data_reg = data_reg.merge(bea_codes, how = 'left')

# Merge with BEA segments
data_reg = data_reg.merge(bea_seg, how = 'inner')

In [4]:
# Create some agg variables
data_reg = data_reg.groupby(['indcode', 'year'])['regindex', 'logreg']\
                   .agg(['mean', 'median'])\
                   .reset_index()

data_reg.columns = [x[0] + x[1] for x in data_reg.columns.to_flat_index()]
data_reg.rename(columns = {
    'regindexmean': 'a1m_regindex',
    'regindexmedian': 'a1med_regindex',
    'logregmean': 'a1m_logreg',
    'logregmedian': 'a1med_logreg',
}, inplace = True)

# Save data
data_reg.to_stata('Data/Intermediate/regindex_out.dta', write_index = False)

# Spreads

Here I do some industry code merging and light aggregating for the industry spread datasets by Glichrist and Zakrajšek

In [5]:
# Import data
data_spread = pd.read_csv('Data/Raw inputs/spr_naics3_q.csv', parse_dates = ['date'])

# Some date stuff
data_spread['year'] = data_spread['date'].dt.year
data_spread['qtr'] = data_spread['date'].dt.quarter

# Keep only 4th quarter
data_spread.query('qtr == 4', inplace = True)

# Reanme and drop columns
data_spread.rename(columns = {'naics3': 'naics'}, inplace = True)
data_spread.drop(columns = ['date'], inplace = True)

In [6]:
# Merge with bea codes
data_spread = data_spread.merge(bea_codes, how = 'left')

# Merge with bea segments
data_spread = data_spread.merge(bea_seg, how = 'inner')

In [7]:
# Some more or less aggregate stuff
data_spread.eval('nb_spavg = nbonds * spr_avg', inplace = True)

data_spread = data_spread.pivot_table(index = ['indcode', 'year'], 
                                      values = ['nbonds', 'nb_spavg'],
                                      aggfunc = np.sum).reset_index()

data_spread.eval('a1m_spread =  nb_spavg / nbonds', inplace = True)

# Keep stuff
data_spread = data_spread.filter(items = ['year', 'indcode', 'a1m_spread'])

# Save data
data_spread.to_stata('Data/Intermediate/spread_data.dta', write_index = False)

# PDII

Here I do some industry code merging and light aggregating for the PDII occupational licensing datasets by Kleiner-Krueger

In [8]:
# Read in data
data_pdii = pd.read_stata('Data/Raw inputs/PDII_RDD_Survey.dta',
                          columns = ['baseid', 'q11', 'q11a', 'industry'])

# Generate new variables
data_pdii['q11ind'] = None
data_pdii.loc[data_pdii['q11'] == '2: no', 'q11ind'] = 0
data_pdii.loc[data_pdii['q11'] == '1: yes', 'q11ind'] = 1

data_pdii['q11aind'] = None
data_pdii.loc[data_pdii['q11a'] == '2: no', 'q11aind'] = 0
data_pdii.loc[data_pdii['q11a'] == '1: yes', 'q11aind'] = 1

# Generate 3 digit naics
data_pdii['naics'] = data_pdii.industry.astype(str).str.slice(0, 3).astype(int)

# Drop stuff
data_pdii.drop(columns = ['industry', 'q11', 'q11a'], inplace = True)

# Make vars numeric
data_pdii = data_pdii.astype(float)

In [9]:
# Merge with bea codes
data_pdii = data_pdii.merge(bea_codes, how = 'left')

# Merge with bea segments
data_pdii = data_pdii.merge(bea_seg, how = 'inner')

In [10]:
# Aggregate across ind_code
data_pdii = data_pdii.pivot_table(index = 'indcode', values = ['q11ind', 'q11aind'],
                                  aggfunc = np.mean).reset_index()

# Rename
data_pdii.rename(columns = {'q11ind': 'a1m_licensed', 'q11aind': 'a1m_licreq'}, inplace = True)

# Save data
data_pdii.to_stata('Data/Intermediate/license_out.dta', write_index = False)

# Bushee

This merges the Bushee dataset with Thomson Reuters 13F.
Bushee dataset is the one from the replication files, while the TR 13F is pulled from WRDS, specifically the `tfn.s34` database. I select the following columns:
- `year` as the year part of `rdate`
- `mgrno`, `shares`, `cusip`, `mgrname`

I filter the date so as to keep only the entries where the month part of `rdate` is 12.

After the merging I also group together some big investment firms. Here I exclude the recoding done in the stat file for Dimensional, because in the fund they assigned it the same number as Blackrock (typo I guess), and because all those firms they wanted to group under Dimensional already have the same number.

In [11]:
# Read in data, replace none
data_bushee = pd.read_stata('Data/Raw inputs/bushee_data_2015.dta')
data_bushee = data_bushee.replace('.', np.nan)

# Drop duplicated mgrno, year, keep max mgrno_v
data_bushee.sort_values('mgrno_v', ascending= False).reset_index(inplace = True)
data_bushee.drop_duplicates(['mgrno', 'year'], keep = 'last', inplace = True)

# Rename, drop
data_bushee = data_bushee.filter(items = ['year', 'mgrno', 'invpermclass'])\
                         .rename(columns = {'invpermclass': 'invclass'})

In [12]:
# Read in Thompson Reuters 13F
select_str = '''
SELECT date_part('year', rdate) AS year, mgrno, shares, cusip, mgrname, no, sole, shared
FROM tfn.s34 
WHERE date_part('month', rdate) = 12
'''

data_13f = db.raw_sql(select_str)

# Drop none cusip
data_13f.dropna(subset = ['cusip'], inplace = True)

# Drop duplicated mgrno&cusip&year -- in 97% of the cases shares are the same, 
# so it doesn't matter. As for the rest - if they don't care, neither do I
data_13f.drop_duplicates(subset = ['mgrno', 'cusip', 'year'], inplace = True)

In [13]:
# Merge Bushee with TR 13F
data_13f = data_13f.merge(data_bushee, how = 'outer')

# Replace invclass NAs with literal NA
data_13f.loc[data_13f.invclass.isna(), 'invclass'] = 'NA'

In [14]:
# Create new variable for mgrno
data_13f['mgrno_mapped'] = data_13f['mgrno']

# Do some recoding
data_13f.loc[data_13f.mgrname.str.contains('BLACKROCK') == True, 'mgrno_mapped'] = 11386
data_13f.loc[data_13f.mgrname.str.contains('CAPITAL RESEARCH') == True, 'mgrno_mapped'] = 12740
data_13f.loc[data_13f.mgrname.str.contains('VANGUARD GROUP') == True, 'mgrno_mapped'] = 90457

fidelity = ["FIDELITY INTERNATIONAL", "FIDELITY INTERNATL LTD", 
    "FIDELITY INTL LTD", "FIDELITY INTL. LTD.", "FIDELITY MANAGEMENT & RESEARCH",
    "FIDELITY MGMT & RES CORP", "FIDELITY MGMT & RESEARCH (US)", "FIDELITY MGMT & RESEARCH CO"]

data_13f.loc[data_13f.mgrname.isin(fidelity), 'mgrno_mapped'] = 27700

state_str = ["STATE STR BK & TRUST CO BOSTON", "STATE STR CORP", 
    "STATE STR CORPORATION", "STATE STR GBL ADVR IRELAND LTD", "STATE STR RESEARCH & MGMT CO",
    "STATE STR RESEARCH & MGMT CO.", "STATE STR RESR & MGMT", "STATE STREET BOSTON CORP",
    "STATE STREET CORP", "STATE STREET RES. & MGMT", "STATE STREET RESR & MGMT"]

data_13f.loc[data_13f.mgrname.isin(state_str), 'mgrno_mapped'] = 81540

In [15]:
# Save what we have up to this point
data_13f.to_stata('Data/Intermediate/bushee_detailed.dta', write_index = False)

## Percentage ownership

Here I compute percentage ownership of each institution in each firm. For that I merge the existing Bushee/TF 13F database with shares outstanding CRSP database.

That database is availible on WRDS as `crspa.msf`. I extract the following variables:
- `permno, permco, cusip8, year, mkt, shrout`

Here `cusip8` is the 8 character substring (starting from 1) of `cusip`, `year` is just the year part of `date` and `mkt` is the product of `prc` and `shrout`. I also filter to only entries with `month = 12`.

After that I add the `gvkey` using the linking table provided by the authors. Finally, I compute the shares.

In [16]:
# Drop some cols
data_13f = data_13f.filter(items = ['cusip', 'year', 'invclass', 'shares'])

# Sum by cusip, year, invclass
data_13f = data_13f.pivot_table(index = ['cusip', 'year', 'invclass'], 
                                values = 'shares', aggfunc = np.sum)\
                   .reset_index()

# Long to wide table, for some reason
data_13f = data_13f.pivot_table(index = ['cusip', 'year'], columns = ['invclass'])
data_13f.columns = data_13f.columns.droplevel(0)
data_13f = data_13f.reset_index().rename(columns = {'cusip': 'cusip8'})

In [17]:
# Read in CRSP stock data
select_str = '''
SELECT date_part('year', date) AS year, permno, permco, shrout,
       shrout * abs(prc) AS mkt, substr(cusip, 1, 8) AS cusip8
FROM crspa.msf
WHERE date_part('month', date) = 12
'''

data_crsp = db.raw_sql(select_str)

In [18]:
# Merge with CRSP
data_own = data_13f.merge(data_crsp, how = 'inner')

# Sort by permco, year and mktcap and keep only largest mktcap within permco, year
data_own = data_own.sort_values(['permco', 'year', 'mkt']).reset_index(drop = True)
data_own.drop_duplicates(subset = ['permco', 'year'], keep = 'last', inplace = True)

In [19]:
# Read in linking table, rename
link_table = pd.read_stata('Data/Raw inputs/linkingTable.dta')
link_table = link_table.rename(columns = {'lpermco': 'permco'})

# Replace NA in linkenddt with today's date (as good as any)
link_table.loc[link_table['linkenddt'].isna(), 'linkenddt'] = pd.to_datetime('today')

In [20]:
# Merge link table, do some filtering
data_own = data_own.merge(link_table, how = 'inner')
data_own.query('linkdt.dt.year <= year & linkenddt.dt.year >= year', inplace = True)

In [21]:
# Sort by gvkey, year, mkt and keep only the largest mkt
data_own = data_own.sort_values(['gvkey', 'year', 'mkt']).reset_index(drop = True)
data_own.drop_duplicates(subset = ['gvkey', 'year'], keep = 'last', inplace = True)

In [22]:
# Finally, compute shares
data_own['sharesinstowned'] = data_own[['DED', 'NA', 'QIX', 'TRA']].sum(axis = 1)
data_own['pctinsown'] = data_own['sharesinstowned']/(1000*data_own['shrout'])
data_own['pctinsown'] = data_own['pctinsown'].clip(upper = 1)
#data_own.loc[data_own.pctinsown.isna() & (data_own.year <= 2013), 'pctinsown'] = 0

for type_s in ['DED', 'NA', 'QIX', 'TRA']:
    pct_tot = data_own[type_s]/(1000*data_own['shrout'])
    pct_ins = data_own[type_s]/data_own['sharesinstowned']
    
    data_own[f'pctsharetot{type_s}'] = pct_tot
    data_own[f'pctshareins{type_s}'] = pct_ins
    
    data_own[f'pctsharetot{type_s}'] = pct_tot.clip(upper = 1)
    data_own.loc[pct_tot.isna() & (data_own.year <= 2013), f'pctsharetot{type_s}'] = 0

    data_own[f'pctshareins{type_s}'] = pct_ins.clip(upper = 1)
    data_own.loc[pct_ins.isna() & (data_own.year <= 2013), f'pctshareins{type_s}'] = 0
    
    data_own.loc[data_own.year == 1980, f'pctshareins{type_s}'] = None
    data_own.loc[data_own.year == 1980, f'pctsharetot{type_s}'] = None


In [23]:
# Keep only some columns
r = re.compile("pct.*")
keep_cols = ['gvkey', 'year', 'cusip8'] + list(filter(r.match, data_own.columns))
data_own = data_own.filter(items = keep_cols)

In [24]:
# Save data
data_own.to_stata('Data/Intermediate/bushee_firmmap.dta')